In [1]:
import sys
import pandas as pd
from datetime import datetime
import re
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import pymorphy2
import warnings
import numpy as np
import matplotlib.pyplot as plt
import logging, datetime, os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#%config Application.log_level="INFO"
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
warnings.simplefilter('ignore')
morph = pymorphy2.MorphAnalyzer()

Стопслова

In [ ]:
#for w in sorted(stopwords):
#     print('u\'%s\','%format(w))

In [3]:
# with open('stopwords.txt','w') as f:
#     for item in stopwords:
#         f.write('%s\n' % item)

In [13]:
#with open('stopwords.txt','r',encoding='cp1251') as f:
#    stopwords = f.read().splitlines()

In [2]:
from stop_words import get_stop_words
import nltk
nltk.download('stopwords')

with open('stopwords.txt','r', encoding='cp1251') as f:
    stopwords_txt = f.read().splitlines() 
    
with open('names.txt','r', encoding='cp1251') as f:
    stopnames = f.read().splitlines()     

stopwords = stopwords_txt + get_stop_words('ru')  + nltk.corpus.stopwords.words('russian') + stopnames

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ivan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Нормализация текстов:

In [4]:
def delete_punctuation(s):
    return ' '.join((re.sub(r'[№"\'-_/.:?!1234567890()%<>;,+#$&\s+]', u' ', s)).split())

def lemmatize(s):    
    return ' '.join([morph.parse(w)[0].normal_form for w in s.split()])

Для словаря:

In [5]:
def get_word_from_dict(dict_name,index):
    return list(dict_name.keys())[list(dict_name.values()).index(int(index))]    

Вывод результатов:

In [6]:
#Топ слов по классам:
def print_top_words_for_class(n,**kwargs):
    
    classname_p =  kwargs.get('classname_p',None)
    
    for c in set(class_word_count.transpose()[0]):
        
        class_name = get_word_from_dict(class_indexes,c)
        
        if classname_p is not None:
            
                if class_name == classname_p:
        
                    for clss,word,cnt in np.array(pd.DataFrame(class_word_count[class_word_count.transpose()[0] == c]).sort_values(by = [2],ascending = False))[:n]:
                        class_name = get_word_from_dict(class_indexes,clss)
                        word = get_word_from_dict(word_indexes,word) 

                        print(class_name,' | ',word,'|',cnt)
        else:
            
            for clss,word,cnt in np.array(pd.DataFrame(class_word_count[class_word_count.transpose()[0] == c]).sort_values(by = [2],ascending = False))[:n]:
                class_name = get_word_from_dict(class_indexes,clss)
                word = get_word_from_dict(word_indexes,word) 

                print(class_name,' | ',word,'|',cnt)

#Больше всего раз встречающиеся слова по классам:            
def print_most_common_word_by_class():        
    for w in np.array(pd_class_word_count):                    
        class_name = get_word_from_dict(class_indexes,w[0])
        word = get_word_from_dict(word_indexes,w[1]) 
        print(class_name,' | ',word,' | ',w[2])    

#Больше всего раз встречающиеся слова в целом:        
def print_most_common_words(n,**kwargs):
    for w in np.array(pd_class_word_count[[1,3]].drop_duplicates().sort_values(by = [3],ascending = False))[:n]:                    
        word = get_word_from_dict(word_indexes,w[0]) 
        
        mode =  kwargs.get('mode',None)
        
        if mode is not None:
            print(word,' | ',w[1])                
        else:
            print('u\'%s\','%format(word))

#Уникальные ключевые слова классов:        
def print_unique_words(**kwargs):
    
    classname_p =  kwargs.get('classname_p',None)

    if classname_p is not None:

        for w in [ w for w in np.array(pd.DataFrame(keywords_filtered).sort_values(by = [0,1])) if get_word_from_dict(class_indexes,w[0]) == classname_p]:                    

            class_name = get_word_from_dict(class_indexes,w[0])
            word = get_word_from_dict(word_indexes,w[1])                                     
            print(class_name,' | ',word)

    else:

         for w in np.array(pd.DataFrame(keywords_filtered).sort_values(by = [0,1])):                    

            class_name = get_word_from_dict(class_indexes,w[0])
            word = get_word_from_dict(word_indexes,w[1])                 
            print(class_name,' | ',word)

#Слова, которые являются ключевыми в нескольких классах:
def print_midex_words():
    for w in np.array(pd.DataFrame(keywords_mixed).sort_values(by = [1])):                    
        class_name = get_word_from_dict(class_indexes,w[0])
        word = get_word_from_dict(word_indexes,w[1]) 
        print(class_name,' | ',word)    

#Мусорные слова - встречаются в большом количестве классов (больше чем в 70% от общего количества классов):    
def print_garbage_words():
    for w in np.array(pd.DataFrame(keywords_garbage).sort_values(by = [0])):                    
        word = get_word_from_dict(word_indexes,w) 
        print('u\'%s\','%format(word))

#Слова, которые появляются в большом моличестве классов        
def print_words_often_apper_in_classes():
    for word,count in keywords_all_counter.most_common():
        print(get_word_from_dict(word_indexes,word),' | ',count)

#Таблица со всеми классами и количеством одинаковых ключевых слов в них         
def print_table_with_same_words_for_classes():
    #Считаем количество одинаковых ключевых слов в классах:

    def get_index_csw(i,j):

        it = 0
        for w in classes_same_words:
            if w[0] == i and w[1] == j:
                return it
                break
            it += 1   
        
    classes_same_words = np.empty((0,3))

    for i in list(class_indexes.values()):
        for j in list(class_indexes.values()):
            if i != j:
                classes_same_words = np.append(classes_same_words,[[i,j,0]],axis = 0)       

    for i in list(class_indexes.values()):
        for w in keywords_mixed[np.array(keywords_mixed).transpose()[0] == i]:
            for c in keywords_mixed[np.array(keywords_mixed).transpose()[1] == w[1]]:
                if i != c[0]:                
                    classes_same_words[get_index_csw(i,c[0])][2] = classes_same_words[get_index_csw(i,c[0])][2] + 1   

    #Считаем количество ключевых слов в каждом классе:                

    pd_class_key_word_sum =  pd.DataFrame(class_word_count[:,:2])
    pd_class_key_word_sum[2] = pd_class_key_word_sum[1].groupby(pd_class_key_word_sum[0]).transform('count')
    pd_class_key_word_sum = pd_class_key_word_sum.drop([1],axis=1).drop_duplicates().reset_index(drop=True)  

    for w in np.array(pd.DataFrame(classes_same_words).sort_values(by = [0,2],ascending = False)):   
        print(get_word_from_dict(class_indexes,w[0]),'(',int(pd_class_key_word_sum[pd_class_key_word_sum[0] == w[0]][2].iat[0]),')','|',get_word_from_dict(class_indexes,w[1]),'(',int(pd_class_key_word_sum[pd_class_key_word_sum[0] == w[1]][2].iat[0]),')','|',int(w[2]))                        
        
#Получаем одинаковые слова в классах       
def get_same_words(class1,class2,**kwargs):
        
    class1_words = keywords_mixed[np.array(keywords_mixed).transpose()[0] == class_indexes.get(class1)][:,1:]
    class2_words = keywords_mixed[np.array(keywords_mixed).transpose()[0] == class_indexes.get(class2)][:,1:]            

    for w in np.intersect1d(class1_words,class2_words):
        
        mode =  kwargs.get('mode',None)
        
        if mode is not None:
            print('u\'%s\','%format(get_word_from_dict(word_indexes,w)))                                    
        else:
            print(get_word_from_dict(word_indexes,w))
            
# Создаем класс "Мусор"                   
# n - количество топовых в классе слов, наличие которых проверяется в тексте
# Если ни одного из этих слов нет, то текст отправляется в "мусор"
def create_data_with_garbage(data,n):        
    
    top_words = []
    
    data_new  = data.copy()
    
    for c in set(class_word_count.transpose()[0]):
        
        class_name = get_word_from_dict(class_indexes,c)    
            
        for clss,word,cnt in np.array(pd.DataFrame(class_word_count[class_word_count.transpose()[0] == c]).sort_values(by = [2],ascending = False))[:n]:            
            word = get_word_from_dict(word_indexes,word) 

            top_words.append(word)
    
    top_words = set(top_words)       
    
    for index,row in data_new.iterrows():                              

        if not top_words.isdisjoint(row['TEXT'].split(' ')):                
            pass            
        else:                
            data_new.set_value(index,'CLASS','Мусор')
            
    return data_new            

## Выделение ключевых слов:

In [6]:
#Инициализация массива даных:
    
data = pd.read_csv('data_clean.csv', encoding='cp1251', delimiter=';')
data = data.dropna(subset=['TEXT'])
#data = data.drop_duplicates().reset_index(drop = True)

#Создаем словари классов и слов:

class_indexes = {}
for i in range(len((set(data['CLASS'])))):
    class_indexes[sorted(set(data['CLASS']))[i]] = i

cv = CountVectorizer(stop_words=stopwords)    
word_count_vector = cv.fit_transform(data['TEXT'].tolist())       
word_indexes = cv.vocabulary_

#Считаем сколько раз слова встречаются в каждом классе

class_word_count = np.empty((0,3))

for class_name in class_indexes:
        
    text_class = data.loc[data['CLASS'] == class_name]['TEXT'].tolist()  
    
    # Инициализируем и подгоняем CV:
        
    cv = CountVectorizer(max_df = 0.5,min_df = 20,stop_words=stopwords)    
    word_count_vector = cv.fit_transform(text_class)           
    
    # Заполняем массивы для подсчета встерчаемости слов в классах:   
    
    class_word_count = np.append(class_word_count,[[class_indexes.get(class_name),word_indexes.get(word),word_count_vector.sum(axis = 0)[0, idx]] for word, idx in cv.vocabulary_.items()],axis=0) 
    
#Делим ключевые слова на группы "Уникальные", "Перемешанные" и "Мусорные" 

keywords_filtered = np.empty((0,2))
keywords_mixed = np.empty((0,2))
keywords_garbage = np.empty((0,1))

class_word_counter = collections.Counter(class_word_count.transpose()[1])
    
for word in set(class_word_count.transpose()[1]):
    
    class_word = class_word_count[class_word_count.transpose()[1] == word][:,:2]
    
    if len(class_word) == 1:
        
        keywords_filtered = np.append(keywords_filtered,class_word,axis=0)   
    
    elif len(class_word) <= int(max(class_word_counter.values())*0.4):
    
        keywords_mixed = np.append(keywords_mixed,class_word,axis=0) 
    
    else:
        
        keywords_garbage = np.append(keywords_garbage,class_word[0][1])   
        
pd_class_word_count =  pd.DataFrame(class_word_count).sort_values(by = [2],ascending = False)
pd_class_word_count[3] = pd_class_word_count[2].groupby(pd_class_word_count[1]).transform('sum')     

In [12]:
# print_top_words_for_class(50,classname_p = 'Электротовары')
# get_same_words('Электротовары','Страхование',mode=1)
# print_table_with_same_words_for_classes()

### Лемматизация

In [21]:
file_path = 'naznach_for_24_classes_1_product_inn_v5.csv'
file_path_new = 'naznach_for_24_classes_1_product_inn_v5_lemmatized.csv'

data = pd.read_csv(file_path, encoding='cp1251', delimiter=';')
data['TEXT'] = data['TEXT'].apply(lambda text : lemmatize(delete_punctuation(text.lower())))
data = data.dropna(subset=['TEXT'])
data.to_csv(file_path_new,sep = ';',index=False)

In [ ]:
#построчная запись в файл

import csv

csv.field_size_limit(sys.maxsize)

file_path = 'naznach_for_24_classes_1_product_inn_v5.csv'
file_path_new = 'naznach_for_24_classes_1_product_inn_v5_lemmatized.csv'

with open(file_path,mode='r') as old_file:
    
    with open(file_path_new,mode='w') as new_file:
    
        old_file_reader = csv.DictReader(old_file)
        
        new_file_writer = csv.writer(new_file,delimiter=';')

        for row in old_file_reader:
            new_file_writer.writerow([lemmatize(delete_punctuation(str(row).lower()))])                        

In [29]:
file_path = 'naznach_for_24_classes_1_product_inn_v5.csv'
file_path_new = 'naznach_for_24_classes_1_product_inn_v5_lemmatized.csv'

data = pd.read_csv(file_path, encoding='cp1251', delimiter=';')
data_new = pd.read_csv(file_path_new, encoding='utf-8', delimiter=';')